In [1]:
# module
import numpy as np
import pandas as pd
import tqdm
from haversine import haversine
from tqdm import tqdm_pandas, tqdm

In [2]:
#load data
car = pd.read_csv('횡단보도_완료.csv')
ped = pd.read_csv('서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding = 'cp949')

In [3]:
# 차 사고
car.columns = ['crosswalk_id', 'car_lat', 'car_lon']

# 횡단보도
ped_xy = ped[['역명', '위도', '경도']]
ped_xy.columns = ['tube', 'ped_lon', 'ped_lat']

In [4]:
car['key'] = 0
ped_xy['key'] = 0

C:\Users\abc\anaconda3\envs\please\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
prod = pd.merge(car, ped_xy, on = 'key', how = 'outer')

In [6]:
lat1 = np.deg2rad(prod['car_lat'])
lat2 = np.deg2rad(prod['ped_lat'])
lon1 = np.deg2rad(prod['car_lon'])
lon2 = np.deg2rad(prod['ped_lon'])
dlat = lat2 - lat1 
dlon = lon2 - lon1 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [7]:
prod['distance'] = c * 6371 * 1000

In [8]:
prod['distance'].describe()

count    673992.000000
mean      19707.395855
std        9334.792510
min          10.847642
25%       12581.132502
50%       19670.727168
75%       26457.119083
max       47463.291607
Name: distance, dtype: float64

In [9]:
prod2 = prod[prod['distance'] <= 500]

In [10]:
howmany = prod2.groupby('crosswalk_id').count().reset_index().iloc[:, 0:2]

In [11]:
howmany

,crosswalk_id,car_lat
0,06-0000000036,1
1,06-0000000037,1
2,06-0000000041,1
3,06-0000000042,1
4,06-0000000054,1
...,...,...
722,06-0000052770,1
723,06-0000052771,1
724,06-0000052819,1
725,06-0000052827,2


In [12]:
howmany.columns = ['crosswalk_id', 'count']

In [13]:
howmany.describe()

,count
count,727.000000
mean,1.306740
std,0.461458
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [14]:
sum(howmany['count'])

950